In [20]:
import ccxt
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

binance = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '15m'
limit = 1000

since_dt = datetime.now(timezone.utc) - timedelta(days=15*3 + 3) # 4 weeks (+3 to avoid weekends if i fetch index, like the dxy, where they dont work on weekends, and i just remove where dxy is Nan)
since = int(since_dt.timestamp() * 1000) # second in milliseconds

all_candles = []

while True:
    candles = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit) # binance api from ccxt
    if not candles:
        break
    all_candles += candles

    since = candles[-1][0] + 1 # update since to last candle timestamp + 1 ms
    time.sleep(0.5)  # avoid hitting rate limits with api

# Convert to DataFrame (table)
df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [21]:
import yfinance as yf

dxy_daily = yf.download(
    'DX-Y.NYB',
    start=df['timestamp'].min().date() - pd.Timedelta(days=3),  # to avoid weekends
    end=df['timestamp'].max().date() + pd.Timedelta(days=3),
    interval='1d',
    progress=False,
    auto_adjust=False
)

# Reset index to move 'Date' to a column
dxy_daily = dxy_daily.reset_index(drop=False)

# Because its multilevel, make it one level (like df)
if isinstance(dxy_daily.columns, pd.MultiIndex):
    dxy_daily.columns = [col[0] for col in dxy_daily.columns]

dxy_daily.rename(columns={'Date': 'date'}, inplace=True)
dxy_daily.rename(columns={'Close': 'dxy_close'}, inplace=True)

dxy_daily["date"] = pd.to_datetime(dxy_daily["date"]).dt.date
df["date"] = pd.to_datetime(df["timestamp"]).dt.date

print(dxy_daily)

df = pd.merge(df, dxy_daily[["date", "dxy_close"]], on="date", how="left")

df['dxy_close'] = df['dxy_close'].ffill()
df = df.dropna(subset=['dxy_close']).reset_index(drop=True)
df.rename(columns={'dxy_close': 'dxy'}, inplace=True)

print(dxy_daily)

          date  Adj Close  dxy_close       High        Low       Open  Volume
0   2025-08-14  98.250000  98.250000  98.320000  97.629997  97.750000       0
1   2025-08-15  97.849998  97.849998  98.209999  97.730003  98.180000       0
2   2025-08-18  98.169998  98.169998  98.190002  97.779999  97.790001       0
3   2025-08-19  98.269997  98.269997  98.320000  97.940002  98.150002       0
4   2025-08-20  98.220001  98.220001  98.440002  98.080002  98.290001       0
5   2025-08-21  98.620003  98.620003  98.680000  98.180000  98.250000       0
6   2025-08-22  97.720001  97.720001  98.830002  97.559998  98.599998       0
7   2025-08-25  98.430000  98.430000  98.540001  97.709999  97.730003       0
8   2025-08-26  98.230003  98.230003  98.559998  98.089996  98.470001       0
9   2025-08-27  98.230003  98.230003  98.730003  98.160004  98.250000       0
10  2025-08-28  97.809998  97.809998  98.230003  97.739998  98.120003       0
11  2025-08-29  97.769997  97.769997  98.129997  97.690002  97.8

In [22]:
import ta
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Short-term (~30min)
df['rsi_6'] = RSIIndicator(df['close'], window=6).rsi()

# Classic (~1hour)
df['rsi_12'] = RSIIndicator(df['close'], window=12).rsi()

macd_indicator = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)

df['macd'] = macd_indicator.macd()                  # MACD line
df['macd_signal'] = macd_indicator.macd_signal()    # Signal line
df['macd_hist'] = macd_indicator.macd_diff()        # Histogram



In [23]:

# df['ema_9'] = df['close'].ewm(span=9, adjust=False).mean()

df['ema_21'] = df['close'].ewm(span=21, adjust=False).mean()

df['sma_50'] = df['close'].rolling(window=50).mean()

df.dropna(subset=['ema_21', 'sma_50'], inplace=True)

df['ema/sma crossover'] = df['ema_21'] - df['sma_50']

df['trends with sma'] = df['close'] / df['sma_50'] - 1

In [24]:
from utils import buy_col, sell_col, define_activity

df['next_high'] = df['high'].shift(-1)
df['next_low'] = df['low'].shift(-1)

df['price change'] = np.log(df['close'] / df['close'].shift(1))
df.dropna(subset=['price change'], inplace=True)

df['buy_index'] = df.apply(buy_col, axis=1)
df['sell_index'] = df.apply(sell_col, axis=1)

df['signal'] = df['buy_index'] - df['sell_index']
scale = df['signal'].abs().median()
df['signal_scaled'] = 0.5 * (np.tanh( (df['buy_index'] - df['sell_index']) / scale ) + 1)

df['activity'] = df.apply(define_activity, axis=1)
# df['activity'] = np.where(df['close'].shift(-5) > df['close'], 'Buy', 'Sell')

df.dropna(subset=['buy_index','sell_index'], inplace=True)


In [25]:
# Add Bollinger Bands
from ta.volatility import BollingerBands

bollinger = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bollinger_hband'] = bollinger.bollinger_hband()
df['bollinger_lband'] = bollinger.bollinger_lband()
df['bollinger_mavg'] = bollinger.bollinger_mavg()
df['bollinger_bandwidth'] = bollinger.bollinger_wband()

# Add Average True Range (ATR)
from ta.volatility import AverageTrueRange

atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14)
df['atr'] = atr.average_true_range()

# Add Bitcoin trading volume trends
df['volume_change'] = df['volume'].pct_change()
df['volume_sma_10'] = df['volume'].rolling(window=10).mean()

# Drop rows with NaN values introduced by rolling calculations
df.dropna(inplace=True)

# Print the updated DataFrame
print(df.head())

             timestamp       open       high        low      close     volume  \
69 2025-08-18 17:15:00  116243.96  116688.00  116243.95  116516.11  274.31192   
70 2025-08-18 17:30:00  116516.11  116800.00  116516.10  116532.67  235.18710   
71 2025-08-18 17:45:00  116532.68  116625.10  116428.98  116428.99  102.21270   
72 2025-08-18 18:00:00  116428.98  116437.44  116300.78  116360.97   74.19460   
73 2025-08-18 18:15:00  116360.97  116458.35  116238.71  116360.98   65.74689   

          date        dxy      rsi_6     rsi_12  ...    signal  signal_scaled  \
69  2025-08-18  98.169998  78.500628  69.316542  ...  0.003301       0.996185   
70  2025-08-18  98.169998  78.874483  69.563374  ... -0.002245       0.022206   
71  2025-08-18  98.169998  69.760605  65.940337  ... -0.002710       0.010263   
72  2025-08-18  98.169998  63.943757  63.570713  ... -0.000354       0.355033   
73  2025-08-18  98.169998  63.944287  63.570923  ...  0.001753       0.950504   

    activity  bollinger_hb

In [26]:
print(df)
df.set_index('timestamp', inplace=True)

# export_df = df[['dxy', 'rsi_6', 'rsi_12', 'ema_21', 'sma_50', 'activity']]
export_df = df.drop(columns=['date', 'next_high', 'next_low', 'buy_index', 'sell_index', 'signal', 'signal_scaled'])
print(export_df)

%store export_df

               timestamp       open       high        low      close  \
69   2025-08-18 17:15:00  116243.96  116688.00  116243.95  116516.11   
70   2025-08-18 17:30:00  116516.11  116800.00  116516.10  116532.67   
71   2025-08-18 17:45:00  116532.68  116625.10  116428.98  116428.99   
72   2025-08-18 18:00:00  116428.98  116437.44  116300.78  116360.97   
73   2025-08-18 18:15:00  116360.97  116458.35  116238.71  116360.98   
...                  ...        ...        ...        ...        ...   
4591 2025-10-04 19:45:00  121887.87  121959.25  121881.00  121959.25   
4592 2025-10-04 20:00:00  121959.25  122079.65  121891.49  121939.96   
4593 2025-10-04 20:15:00  121939.97  122057.28  121939.96  122057.28   
4594 2025-10-04 20:30:00  122057.28  122057.28  121986.29  122029.48   
4595 2025-10-04 20:45:00  122029.48  122061.83  121892.89  121892.89   

         volume        date        dxy      rsi_6     rsi_12  ...    signal  \
69    274.31192  2025-08-18  98.169998  78.500628  69.31